In [1]:
from __future__ import annotations
import asyncio
import os
import datetime as _dt
from typing import Any, Dict, List, Optional
from dotenv import load_dotenv
from mcp.server.fastmcp import FastMCP
import mcp.types as types
from py_clob_client.client import ClobClient, ApiCreds
from py_clob_client.clob_types import BookParams
from py_clob_client.constants import POLYGON
import time
import requests
from datetime import datetime

In [2]:
load_dotenv()

True

In [3]:
def _new_clob() -> ClobClient:
    """Return an authenticated py‑clob client using env vars."""
    host = os.getenv("CLOB_HOST", "https://clob.polymarket.com")
    key = os.getenv("PK")  # Polygon private key
    api_key = os.getenv("CLOB_API_KEY")
    api_secret = os.getenv("CLOB_SECRET")
    passphrase = os.getenv("CLOB_PASS_PHRASE")

    client = ClobClient(host, key=key, chain_id=POLYGON)
    client.set_api_creds(dict(apiKey=api_key, secret=api_secret, passphrase=passphrase))
    return client

In [4]:
client = _new_clob()

In [5]:
def list_all_prediction_markets(query: str) -> List[Dict[str, Any]]:
    """
    Fetch all CLOB markets, filter those whose description/question/slug
    contains `query` (case-insensitive) and return a list of dicts with market details.
    """
    client = _new_clob()
    search = query.lower().strip()

    cursor = ""
    results: List[Dict[str, Any]] = []

    while True:
        # 1) fetch one page
        page = client.get_markets(next_cursor=cursor)
        cursor = page.get("next_cursor", "")

        for m in page.get("data", []):
            # 2) collect searchable fields
            desc = (m.get("description") or "").lower()
            ques = (m.get("question") or "").lower()
            slug = (m.get("market_slug") or "").lower()

            # 3) skip if query not in any
            if all(search not in field for field in (desc, ques, slug)):
                continue

            # 4) build dynamic outcomes & prices lists
            outcomes: List[str] = []
            prices: List[float] = []
            for token in m.get("tokens", []):
                out = token.get("outcome")
                pr = token.get("price")
                if out is None or pr is None:
                    continue
                outcomes.append(out)
                try:
                    prices.append(float(pr))
                except (TypeError, ValueError):
                    prices.append(None)

            # 5) append the summary
            results.append(
                {
                    "condition_id": m.get("condition_id", None),
                    "question_id": m.get("question_id", None),
                    "slug": m.get("market_slug"),
                    "question": m.get("question"),
                    "description": (m.get("description") or ""),
                    "outcomes": outcomes,
                    "prices": prices,
                    "volume": m.get("volume"),
                    "active": m.get("active"),
                    "closed": m.get("closed"),
                    "startDate": m.get("game_start_time") or m.get("start_date_iso"),
                    "endDate": m.get("end_date_iso"),
                }
            )

        # 6) stop when no more pages
        if not cursor or cursor == "LTE=":
            break

    return results

In [6]:
if __name__ == "__main__":
    markets = list_all_prediction_markets("India")
    for m in markets:
        print(f"Question: {m['question']}")
        print(f"Description: {m['description']}")
        print(f"Slug: {m['slug']}")
        print(f"Condition ID: {m['condition_id']}")
        print(f"Question ID: {m['question_id']}")
        print(f"Volume: {m['volume']}")
        print(f"Active: {m['active']}")
        print(f"Closed: {m['closed']}")
        print(f"Start Date: {m['startDate']}")
        print(f"End Date: {m['endDate']}")
        for o, p in zip(m["outcomes"], m["prices"]):
            print(f"Outcome: {o}, Price: {p}")
        print()

Question: NBA: Lakers vs. Pacers (02/02/2023)
Description: In the upcoming NBA game, scheduled for February 2:

If the Los Angeles Lakers win, the market will resolve to “Lakers”.

If the Indiana Pacers win, the market will resolve to “Pacers”.

If this game is not completed by April 2, 2023 (11:59:59 PM ET), or is definitively cancelled before that date, this market will resolve 50-50.

Slug: nba-lakers-vs-pacers-02022023
Condition ID: 0x3960e6ba226b18e349a586703c942d334325f3c026d179a7bcf8a524edaa40f4
Question ID: 0x5ba4abc14e410397f08dc96a4255c54ea49e23cd4a646d6aea57206334865f53
Volume: None
Active: True
Closed: True
Start Date: 2023-02-03T00:00:00Z
End Date: 2023-02-02T00:00:00Z
Outcome: Lakers, Price: 0.0103
Outcome: Pacers, Price: 0.9897

Question: NBA: Cavaliers vs. Pacers (02/05/2023)
Description: In the upcoming NBA game, scheduled for February 5:

If the Cleveland Cavaliers win, the market will resolve to “Cavaliers”.

If the Indiana Pacers win, the market will resolve to “Pac

In [7]:
def list_prediction_market_orderbook(
    market_id: str,
    snapshot_time: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Fetches, for each outcome in the given market:
      - best bid, best ask, and spread
      - full bid ladder and ask ladder

    Returns a dict:
    {
      "market_id": ...,
      "question" : ...,
      "orderbooks": {
        "<outcome>": {
          "best_bid": float,
          "best_ask": float,
          "spread": float,
          "bids":  [ {"price": str, "size": str}, ... ],
          "asks":  [ {"price": str, "size": str}, ... ],
        },
        ...
      }
    }

    Note: historical snapshots aren’t supported by the public REST API,
    so `snapshot_time` is accepted but ignored (always returns the live book).
    """
    # --- Client init (L2 API-Key auth) ---
    host = "https://clob.polymarket.com"
    eth_key = os.getenv("PK", "")
    creds = ApiCreds(
        api_key=os.getenv("CLOB_API_KEY", ""),
        api_secret=os.getenv("CLOB_SECRET", ""),
        api_passphrase=os.getenv("CLOB_PASS_PHRASE", ""),
    )
    client = ClobClient(host, key=eth_key, chain_id=137, creds=creds)

    # --- Fetch market metadata ---
    market = client.get_market(condition_id=market_id)
    question = market["question"]

    result: Dict[str, Any] = {
        "market_id": market_id,
        "question": question,
        "orderbooks": {},
    }

    # --- For each outcome token, fetch its book ---
    for tok in market["tokens"]:
        token_id = tok["token_id"]
        outcome = tok["outcome"]

        # (ignore snapshot_time because not available via REST)
        book = client.get_order_book(token_id=token_id)

        # book.bids and book.asks are lists of OrderSummary objects
        bids = book.bids
        asks = book.asks

        # best bid/ask/spread
        best_bid = float(bids[0].price) if bids else None
        best_ask = float(asks[0].price) if asks else None
        spread = (
            (best_ask - best_bid)
            if (best_bid is not None and best_ask is not None)
            else None
        )

        # build the ladders as plain dicts
        bid_list = [{"price": lvl.price, "size": lvl.size} for lvl in bids]
        ask_list = [{"price": lvl.price, "size": lvl.size} for lvl in asks]

        result["orderbooks"][outcome] = {
            "best_bid": best_bid,
            "best_ask": best_ask,
            "spread": spread,
            "bids": bid_list,
            "asks": ask_list,
        }

    return result

In [9]:
if __name__ == "__main__":
    load_dotenv()  # load PK, CLOB_* from .env

    market_id = os.getenv(
        "MARKET_ID",
        "0x660e04d89d46a4d2a3a372bad6c82fd7a4a37e09e02816df0084dd540062deaf",
    )
    snapshot_iso = os.getenv("SNAPSHOT_TIME", None)

    ob = list_prediction_market_orderbook(market_id, snapshot_iso)

    # Pretty-print it:
    print(f"\nMarket: {ob['question']}  (ID: {ob['market_id']})\n")
    for outcome, data in ob["orderbooks"].items():
        print(f"→ Outcome: {outcome}")
        print(
            f"    Best bid = {data['best_bid']}, ask = {data['best_ask']}, spread = {data['spread']}"
        )
        print("    Full bids:")
        for lvl in data["bids"]:
            print(f"      • {lvl['price']} @ {lvl['size']}")
        print("    Full asks:")
        for lvl in data["asks"]:
            print(f"      • {lvl['price']} @ {lvl['size']}")
        print()

PolyApiException: PolyApiException[status_code=404, error_message={'error': 'No orderbook exists for the requested token id'}]

In [ ]:
#!/usr/bin/env python3
"""
Interactive Polymarket line-chart (Yes = blue, No = red) with
a dropdown for quick interval switching.

Usage:
    python market_plot.py        # runs demo
"""
import requests
import plotly.graph_objects as go
import plotly.io as pio
from typing import Any, Dict, List, Optional, Tuple

# --------------------------------------------------------------------
# Plotly renderer — change to suit your environment:
#   "browser"  = opens default web-browser
#   "svg"/"png"= static images
#   leave commented if you already work inside Jupyter with nbformat
# --------------------------------------------------------------------
pio.renderers.default = "browser"

CLOB_REST = "https://clob.polymarket.com"
VALID_INTERVALS = ["max", "1m", "1w", "1d", "6h", "1h"]


# --------------------------------------------------------------------
# Internal helper
# --------------------------------------------------------------------
def _fetch_interval(
    market_id: str,
    interval: str,
    fidelity: int,
    start_ts: Optional[int],
    end_ts: Optional[int],
) -> Dict[str, Any]:
    mresp = requests.get(f"{CLOB_REST}/markets/{market_id}")
    mresp.raise_for_status()
    market = mresp.json()
    question = market["question"]

    raw: Dict[str, Dict[int, float]] = {}
    all_ts = set()

    for tok in market["tokens"]:
        token_id = tok["token_id"]
        outcome = tok["outcome"]

        params: Dict[str, Any] = {"market": token_id, "fidelity": fidelity}
        if start_ts is not None or end_ts is not None:
            if start_ts is not None:
                params["startTs"] = start_ts
            if end_ts is not None:
                params["endTs"] = end_ts
        else:
            params["interval"] = interval

        hresp = requests.get(f"{CLOB_REST}/prices-history", params=params)
        hresp.raise_for_status()
        history = hresp.json()["history"]

        mapping = {pt["t"]: pt["p"] for pt in history}
        raw[outcome] = mapping
        all_ts.update(mapping.keys())

    ts_sorted = sorted(all_ts)
    series = {
        outcome: [raw[outcome].get(ts, None) for ts in ts_sorted] for outcome in raw
    }
    return {
        "market_id": market_id,
        "question": question,
        "timestamps": ts_sorted,
        "series": series,
        "interval": interval,
    }


# --------------------------------------------------------------------
# Main public function
# --------------------------------------------------------------------
def list_prediction_market_graph(
    market_id: str,
    interval: str = "1d",
    fidelity: int = 50,
    start_ts: Optional[int] = None,
    end_ts: Optional[int] = None,
    build_chart: bool = True,
) -> Tuple[Dict[str, Any], Optional[go.Figure]]:
    """
    Returns:
        data_dict, plotly_figure
    If build_chart=False, the second element is None.
    """
    if interval not in VALID_INTERVALS:
        raise ValueError(f"interval must be one of {VALID_INTERVALS}")

    # Pull data for all intervals so dropdown is instant
    graphs = {
        iv: _fetch_interval(market_id, iv, fidelity, start_ts, end_ts)
        for iv in VALID_INTERVALS
    }
    base = graphs[interval]

    if not build_chart:
        return base, None

    fig = go.Figure()
    for iv, g in graphs.items():
        visible = iv == interval
        x_ms = [ts * 1000 for ts in g["timestamps"]]  # Plotly wants ms

        fig.add_trace(
            go.Scatter(
                x=x_ms,
                y=g["series"]["Yes"],
                mode="lines",
                name="Yes",
                line=dict(color="blue"),
                visible=visible,
            )
        )
        fig.add_trace(
            go.Scatter(
                x=x_ms,
                y=g["series"]["No"],
                mode="lines",
                name="No",
                line=dict(color="red"),
                visible=visible,
            )
        )

    # Visibility masks for dropdown
    n_per_iv = 2
    visibility = []
    for iv in VALID_INTERVALS:
        mask = []
        for other_iv in VALID_INTERVALS:
            mask.extend([other_iv == iv, other_iv == iv])
        visibility.append(mask)

    fig.update_layout(
        title=base["question"],
        xaxis_title="Timestamp (UTC)",
        yaxis_title="Price",
        updatemenus=[
            dict(
                active=VALID_INTERVALS.index(interval),
                buttons=[
                    dict(label=iv, method="update", args=[{"visible": visibility[i]}])
                    for i, iv in enumerate(VALID_INTERVALS)
                ],
                x=1.12,
                y=1.15,
            )
        ],
        legend=dict(x=0.01, y=0.99),
    )
    return base, fig

In [ ]:
if __name__ == "__main__":
    data, chart = list_prediction_market_graph(
        market_id="0x660e04d89d46a4d2a3a372bad6c82fd7a4a37e09e02816df0084dd540062deaf"
    )

    # --------- quick text summary ---------
    print(f"Market: {data['question']}  (ID: {data['market_id']})")
    print(f"Total timestamps: {len(data['timestamps'])}")
    print("First 5 timestamps:", data["timestamps"][:])
    print("→ Yes:", data["series"]["Yes"][:])
    print("→ No :", data["series"]["No"][:])
    # --------------------------------------

    chart.show()

Market: Will Buffett say "Inflation" 4+ times during May 3 Shareholders Meeting?  (ID: 0x660e04d89d46a4d2a3a372bad6c82fd7a4a37e09e02816df0084dd540062deaf)
Total timestamps: 30
First 5 timestamps: [1745649006, 1745652006, 1745655005, 1745658005, 1745661005, 1745664006, 1745667006, 1745670006, 1745673005, 1745676005, 1745679006, 1745682006, 1745685006, 1745688005, 1745691006, 1745694006, 1745697006, 1745700005, 1745703006, 1745706006, 1745709005, 1745712005, 1745715005, 1745718005, 1745721005, 1745724005, 1745727005, 1745730005, 1745733005, 1745734685]
→ Yes: [0.795, 0.67, 0.67, 0.665, 0.66, 0.66, 0.66, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.56, 0.555, 0.555, 0.555, 0.555, 0.555, 0.555, 0.555]
→ No : [0.205, 0.33, 0.33, 0.335, 0.34, 0.34, 0.34, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.44, 0.445, 0.445, 0.445, 0.445, 0.445, 0.445, 0.445]


In [13]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions

In [14]:
# Create a Chroma Client
chroma_client = chromadb.Client(
    Settings(
        persist_directory="./_chroma_data",
        chroma_db_impl="duckdb+parquet",
    )
)

ValueError: An instance of Chroma already exists for ephemeral with different settings